In [1]:
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon.git

Cloning into 'Qualcomm-DL-Hackathon'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 30.68 MiB | 25.83 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
import zipfile
import os
import shutil

def extract_all_files(zip_dir, output_dir):
    """
        This function is used to extract all the images from images part-1.zip and images part-2.zip to all_images folder
    """
    if os.path.exists(zip_dir):
      # Get the zip files in train directory
      files = os.listdir(zip_dir)
      zip_files = [f for f in files if f.endswith('.zip')]

      if not os.path.exists(output_dir):
        os.mkdir(output_dir)

      # Extract each zip file
      for zip_file in zip_files:
          with zipfile.ZipFile(os.path.join(zip_dir, zip_file), 'r') as zip_ref:
              for member in zip_ref.namelist():
                  # Extract each file to the all_images directory without creating subfolders
                  filename = os.path.basename(member)
                  if filename:
                      source = zip_ref.open(member)
                      target = open(os.path.join(output_dir, filename), "wb")
                      with source, target:
                          shutil.copyfileobj(source, target)
    else:
      print("f{zip_dir} doesn't exits")

In [3]:
# Define the directory containing the zip files and the all_images directory
train_directory = '/content/Qualcomm-DL-Hackathon/train/'
all_images_directory = os.path.join(train_directory , 'all_images')

# Extract all files from the zip files to the all_images directory
extract_all_files(train_directory, all_images_directory)

print(f"All files have been extracted to {all_images_directory}.")

All files have been extracted to /content/Qualcomm-DL-Hackathon/train/all_images.


In [4]:
from tensorflow import keras
from keras.models import Sequential
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.layers import Dense
import pandas as pd

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.applications.vgg16 import decode_predictions
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, Reshape, Activation

from sklearn.metrics import log_loss
from tqdm import tqdm


In [5]:
train = pd.read_csv('/content/Qualcomm-DL-Hackathon/train/train.csv')

train_img=[]
for i in tqdm(range(train.shape[0])):
  temp_img = image.load_img('/content/Qualcomm-DL-Hackathon/train/all_images/'+train['image_names'][i], target_size=(128,128))
  temp_img=image.img_to_array(temp_img)

  train_img.append(temp_img)


100%|██████████| 1646/1646 [00:03<00:00, 442.06it/s]


In [6]:
from keras.models import Model

def vgg16_model(img_rows, img_cols, channel=1, num_classes=None):

    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, channel))
    # Freeze the layers which you don't want to train
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom layers on top of the base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)

    # Create the new model
    model = Model(inputs=base_model.input, outputs=x)

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [10]:
train_y = train['emergency_or_not'].values
le = LabelEncoder()

train_y = le.fit_transform(train_y)

train_y=to_categorical(train_y)
train_y=np.array(train_y)

from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid=train_test_split(train_img,train_y,test_size=0.2, random_state=42)

# Example to fine-tune on 3000 samples from Cifar10

img_rows, img_cols = 128, 128 # Resolution of inputs
channel = 3
num_classes = 2
batch_size = 32
nb_epoch = 15

X_train = np.stack(X_train)
X_valid = np.stack(X_valid)
Y_train = np.stack(Y_train)
Y_valid = np.stack(Y_valid)

# Load our model
model = vgg16_model(img_rows, img_cols, channel, num_classes)

In [11]:
model.fit(X_train, Y_train,batch_size=batch_size,epochs=nb_epoch,shuffle=True,verbose=1,validation_data=(X_valid, Y_valid))

# Make predictions
predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)

c = np.argmax(predictions_valid, axis=1)
c

Epoch 1/15
83/83 ━━━━━━━━━━━━━━━━━━━━ 35s 124ms/step - accuracy: 0.7398 - loss: 24.6171 - val_accuracy: 0.8939 - val_loss: 1.5064
Epoch 2/15
83/83 ━━━━━━━━━━━━━━━━━━━━ 27s 89ms/step - accuracy: 0.9421 - loss: 0.5981 - val_accuracy: 0.8879 - val_loss: 0.6155
Epoch 3/15
83/83 ━━━━━━━━━━━━━━━━━━━━ 8s 91ms/step - accuracy: 0.9431 - loss: 0.4983 - val_accuracy: 0.9152 - val_loss: 0.3292
Epoch 4/15
83/83 ━━━━━━━━━━━━━━━━━━━━ 9s 106ms/step - accuracy: 0.9807 - loss: 0.0801 - val_accuracy: 0.9061 - val_loss: 0.4040
Epoch 5/15
83/83 ━━━━━━━━━━━━━━━━━━━━ 10s 103ms/step - accuracy: 0.9693 - loss: 0.1430 - val_accuracy: 0.9152 - val_loss: 0.3643
Epoch 6/15
83/83 ━━━━━━━━━━━━━━━━━━━━ 8s 102ms/step - accuracy: 0.9667 - loss: 0.1257 - val_accuracy: 0.9061 - val_loss: 0.3831
Epoch 7/15
83/83 ━━━━━━━━━━━━━━━━━━━━ 7s 89ms/step - accuracy: 0.9772 - loss: 0.0616 - val_accuracy: 0.9091 - val_loss: 0.5931
Epoch 8/15
83/83 ━━━━━━━━━━━━━━━━━━━━ 7s 88ms/step - accuracy: 0.9699 - loss: 0.0821 - val_accuracy: 0.

array([1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [12]:
test = pd.read_csv('/content/Qualcomm-DL-Hackathon/test.csv')

test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('/content/Qualcomm-DL-Hackathon/train/all_images/'+test['image_names'][i], target_size=(128,128))
    img = image.img_to_array(img)
    test_image.append(img)

test_image = np.stack(test_image)
test_image.shape

# Make predictions
predictions_valid2 = model.predict(test_image, batch_size=batch_size, verbose=1)

predictions_valid2

d = np.argmax(predictions_valid2, axis=1)
d

100%|██████████| 706/706 [00:00<00:00, 1054.37it/s]


45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 129ms/step


array([0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,

In [13]:
# creating submission file
sample = pd.read_csv('/content/Qualcomm-DL-Hackathon/test.csv')
sample['emergency_or_not'] = d
sample.to_csv('/content/Qualcomm-DL-Hackathon/submission.csv', header=True, index=False)